In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import shapefile as shp
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import sys
sys.path.append('../scripts')
from preprocess_script import count_outliers

In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Dataset Joining")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("OFF")

24/09/11 07:41:29 WARN Utils: Your hostname, Anujs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.132.154 instead (on interface en0)
24/09/11 07:41:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/11 07:41:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Dataset downloaded from https://data.gov.au/dataset/ds-dga-2c79581f-600e-4560-80a8-98adb1922dfc/details?q=correspondence%20asgs, data dictionary also in the same.

In [10]:
correspondence_df = spark.read.csv('../data/tables/correspondence/CG_POSTCODE_2021_SA2_2021.csv', header=True)
correspondence_df.show()

+--------+-------------+----------------+-------------+------------------------------+-------------------------+--------------+
|POSTCODE|SA2_CODE_2021|   SA2_NAME_2021|RATIO_FROM_TO|INDIV_TO_REGION_QLTY_INDICATOR|OVERALL_QUALITY_INDICATOR|BMOS_NULL_FLAG|
+--------+-------------+----------------+-------------+------------------------------+-------------------------+--------------+
|    0800|    701011002|     Darwin City|            1|                          Good|                     Poor|             0|
|    0810|    701021018|         Jingili|    0.0534036|                          Poor|                     Poor|             0|
|    0810|    701021029|         Wanguri|     0.053635|                          Poor|                     Poor|             0|
|    0810|    701021024|            Moil|     0.058218|                          Poor|                     Poor|             0|
|    0810|    701021028|         Wagaman|    0.0589462|                          Poor|                  

In [11]:
preferred_SA2 = correspondence_df.groupBy('POSTCODE').agg(F.max('RATIO_FROM_TO'))
preferred_SA2.show()

+--------+------------------+
|POSTCODE|max(RATIO_FROM_TO)|
+--------+------------------+
|    NULL|                 1|
|    0800|                 1|
|    0810|         0.1745598|
|    0812|          3.00E-07|
|    0820|          1.90E-06|
|    0822|          9.07E-05|
|    0828|          3.00E-06|
|    0829|         0.9888128|
|    0830|          3.30E-06|
|    0832|         0.3493221|
|    0834|                 1|
|    0835|          0.979292|
|    0836|          7.00E-07|
|    0837|                 1|
|    0838|                 1|
|    0839|                 1|
|    0840|                 1|
|    0841|                 1|
|    0845|                 1|
|    0846|                 1|
+--------+------------------+
only showing top 20 rows

+--------+------------------+
|POSTCODE|max(RATIO_FROM_TO)|
+--------+------------------+
|    NULL|                 1|
|    0800|                 1|
|    0810|         0.1745598|
|    0812|          3.00E-07|
|    0820|          1.90E-06|
|    0822|    

In [12]:
preferred_SA2 = preferred_SA2.withColumnRenamed("max(RATIO_FROM_TO)", "RATIO_FROM_TO")

In [17]:
preferred_SA2.show()
preferred_SA2.count()
preferred_SA2.dropna()
preferred_SA2.count()

+--------+-------------+
|POSTCODE|RATIO_FROM_TO|
+--------+-------------+
|    NULL|            1|
|    0800|            1|
|    0810|    0.1745598|
|    0812|     3.00E-07|
|    0820|     1.90E-06|
|    0822|     9.07E-05|
|    0828|     3.00E-06|
|    0829|    0.9888128|
|    0830|     3.30E-06|
|    0832|    0.3493221|
|    0834|            1|
|    0835|     0.979292|
|    0836|     7.00E-07|
|    0837|            1|
|    0838|            1|
|    0839|            1|
|    0840|            1|
|    0841|            1|
|    0845|            1|
|    0846|            1|
+--------+-------------+
only showing top 20 rows

+--------+-------------+
|POSTCODE|RATIO_FROM_TO|
+--------+-------------+
|    NULL|            1|
|    0800|            1|
|    0810|    0.1745598|
|    0812|     3.00E-07|
|    0820|     1.90E-06|
|    0822|     9.07E-05|
|    0828|     3.00E-06|
|    0829|    0.9888128|
|    0830|     3.30E-06|
|    0832|    0.3493221|
|    0834|            1|
|    0835|     0.979292|

2643

In [18]:
preferred_SA2 = preferred_SA2.join(correspondence_df, on=["POSTCODE", "RATIO_FROM_TO"], how = "left")

In [19]:
preferred_SA2.show()

+--------+-------------+-------------+--------------------+------------------------------+-------------------------+--------------+
|POSTCODE|RATIO_FROM_TO|SA2_CODE_2021|       SA2_NAME_2021|INDIV_TO_REGION_QLTY_INDICATOR|OVERALL_QUALITY_INDICATOR|BMOS_NULL_FLAG|
+--------+-------------+-------------+--------------------+------------------------------+-------------------------+--------------+
|    NULL|            1|         NULL|                NULL|                          NULL|                     NULL|          NULL|
|    0800|            1|    701011002|         Darwin City|                          Good|                     Poor|             0|
|    0810|    0.1745598|    701021021|          Lyons (NT)|                          Poor|                     Poor|             0|
|    0812|     3.00E-07|    701021021|          Lyons (NT)|                          Poor|                     Poor|             1|
|    0820|     1.90E-06|    701021015|      Charles Darwin|                 

In [20]:
preferred_SA2.select('POSTCODE', 'SA2_CODE_2021')

POSTCODE,SA2_CODE_2021
NULL,NULL
0800,701011002
0810,701021021
0812,701021021
0820,701021015
0822,701021015
0828,701021017
0829,701031031
0830,701031033
0832,701041043


In [23]:
# Removing Null values
preferred_SA2 = preferred_SA2.na.drop(how = 'any')

In [41]:
# Information on consumer
consumer_user_detail = spark.read.parquet("../data/tables/part_1/consumer_user_details.parquet")
consumer = spark.read.option("delimiter" , "|").csv("../data/tables/part_1/tbl_consumer.csv", header = "True")

# Information on consumer's fraud probability
consumer_fraud_prob = spark.read.csv("../data/tables/part_1/consumer_fraud_probability.csv", header = "True")

In [42]:
consumer_user_detail.show()

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|      1|    1195503|
|      2|     179208|
|      3|    1194530|
|      4|     154128|
|      5|     712975|
|      6|     407340|
|      7|     511685|
|      8|     448088|
|      9|     650435|
|     10|    1058499|
|     11|     428325|
|     12|    1494640|
|     13|    1146717|
|     14|    1343547|
|     15|    1463076|
|     16|    1356405|
|     17|    1331093|
|     18|      80965|
|     19|    1226530|
|     20|    1390367|
+-------+-----------+
only showing top 20 rows

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|      1|    1195503|
|      2|     179208|
|      3|    1194530|
|      4|     154128|
|      5|     712975|
|      6|     407340|
|      7|     511685|
|      8|     448088|
|      9|     650435|
|     10|    1058499|
|     11|     428325|
|     12|    1494640|
|     13|    1146717|
|     14|    1343547|
|     15|    1463076|
|     16|    1356405|
|     17|    1331093|
|     

In [43]:
consumer.show()

+-----------------+--------------------+-----+--------+-----------+-----------+
|             name|             address|state|postcode|     gender|consumer_id|
+-----------------+--------------------+-----+--------+-----------+-----------+
| Yolanda Williams|413 Haney Gardens...|   WA|    6935|     Female|    1195503|
|       Mary Smith|     3764 Amber Oval|  NSW|    2782|     Female|     179208|
|    Jill Jones MD|  40693 Henry Greens|   NT|     862|     Female|    1194530|
|  Lindsay Jimenez|00653 Davenport C...|  NSW|    2780|     Female|     154128|
|Rebecca Blanchard|9271 Michael Mano...|   WA|    6355|     Female|     712975|
|    Karen Chapman|2706 Stewart Oval...|  NSW|    2033|     Female|     407340|
|     Andrea Jones|   122 Brandon Cliff|  QLD|    4606|     Female|     511685|
| Stephen Williams|6804 Wright Crest...|   WA|    6056|       Male|     448088|
|  Stephanie Reyes|5813 Denise Land ...|  NSW|    2482|     Female|     650435|
| Jillian Gonzales|461 Ryan Common S...|

In [44]:
consumer_fraud_prob.show()

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
|  16556|    2022-02-20|89.65663294494827|
|  10278|    2021-09-28|83.59136689427714|
|  15790|    2021-12-30|71.77065889280253|
|   5233|    2021-08-29|85.87123303878818|
|    230|    2021-08-28|86.28328808934151|
|  13601|    2021-12-26|83.13696487489679|
|   6383|    2021-09-15| 66.2676451623754|
|   3513|    2022-02-27|75.16981192247916|
|  18658|    2021-10-19|82.98609082999361|
|   5965|    2021-11-14|69.37164467869053|
|  18714|    2021-11-14|83.78813794627237|
|  22957|    2022-02-12|82.79065699075498|
|  20118|    2021-09-05|80.34030486265003|
|   6436|    2021-12-24|84.81618344606828|
|  17900|    2022-02-25|92.73262811161372|
+-------+--

In [45]:
consumer = consumer.join(consumer_user_detail, on = "consumer_id", how="left")

In [46]:
consumer = consumer.join(consumer_fraud_prob, on="user_id", how="left")

In [47]:
consumer.show()

+-------+-----------+-----------------+--------------------+-----+--------+-----------+--------------+------------------+
|user_id|consumer_id|             name|             address|state|postcode|     gender|order_datetime| fraud_probability|
+-------+-----------+-----------------+--------------------+-----+--------+-----------+--------------+------------------+
|      1|    1195503| Yolanda Williams|413 Haney Gardens...|   WA|    6935|     Female|    2022-02-20| 9.805431136520959|
|      2|     179208|       Mary Smith|     3764 Amber Oval|  NSW|    2782|     Female|    2021-08-30| 9.599513915425788|
|      2|     179208|       Mary Smith|     3764 Amber Oval|  NSW|    2782|     Female|    2021-09-25|10.069850934775245|
|      3|    1194530|    Jill Jones MD|  40693 Henry Greens|   NT|     862|     Female|    2021-11-03| 8.300636455314633|
|      4|     154128|  Lindsay Jimenez|00653 Davenport C...|  NSW|    2780|     Female|    2021-10-09| 9.633302411090419|
|      5|     712975|Reb

In [51]:
consumer.schema

StructType([StructField('user_id', LongType(), True), StructField('consumer_id', StringType(), True), StructField('name', StringType(), True), StructField('address', StringType(), True), StructField('state', StringType(), True), StructField('postcode', StringType(), True), StructField('gender', StringType(), True), StructField('order_datetime', StringType(), True), StructField('fraud_probability', StringType(), True)])

In [52]:
from pyspark.sql.types import DoubleType
consumer = consumer.withColumn('fraud_probability', F.col('fraud_probability').cast(DoubleType()))

In [53]:
# Filling 0 for fraud probability null.
consumer = consumer.fillna(0, subset=['fraud_probability'])

In [54]:
max_fraud = consumer.groupby('user_id').agg(F.max('fraud_probability'))

In [55]:
max_fraud.show()

+-------+----------------------+
|user_id|max(fraud_probability)|
+-------+----------------------+
|     26|    10.604536233608997|
|     29|    15.492409971599415|
|    474|     24.74251000337883|
|    964|    10.000639641611324|
|   1677|     10.69910130025277|
|   1697|                   0.0|
|   1806|                   0.0|
|   1950|    16.924789438758555|
|   2040|    19.150047883546396|
|   2214|      9.06669796313919|
|   2250|      25.5950368734215|
|   2453|                   0.0|
|   2509|     17.64221987321122|
|   2529|     16.84817579923174|
|   2927|    14.570508610444977|
|   3091|    15.428265394555776|
|   3506|    30.199921043599648|
|   3764|      9.72214044955335|
|   4590|     18.88641597368393|
|   4823|                   0.0|
+-------+----------------------+
only showing top 20 rows

+-------+----------------------+
|user_id|max(fraud_probability)|
+-------+----------------------+
|     26|    10.604536233608997|
|     29|    15.492409971599415|
|    474|     24.

In [56]:
consumer = consumer.join(max_fraud, on='user_id', how='left')
consumer.show()

+-------+-----------+-----------------+--------------------+-----+--------+-----------+--------------+------------------+----------------------+
|user_id|consumer_id|             name|             address|state|postcode|     gender|order_datetime| fraud_probability|max(fraud_probability)|
+-------+-----------+-----------------+--------------------+-----+--------+-----------+--------------+------------------+----------------------+
|      6|     407340|    Karen Chapman|2706 Stewart Oval...|  NSW|    2033|     Female|    2021-12-12|10.459280127078758|    10.459280127078758|
|      7|     511685|     Andrea Jones|   122 Brandon Cliff|  QLD|    4606|     Female|          NULL|               0.0|                   0.0|
|      9|     650435|  Stephanie Reyes|5813 Denise Land ...|  NSW|    2482|     Female|    2021-11-17| 8.531261989227714|     10.58055311139687|
|      9|     650435|  Stephanie Reyes|5813 Denise Land ...|  NSW|    2482|     Female|    2021-12-13| 10.58055311139687|     10.5

In [57]:
consumer = consumer.select('user_id', 'consumer_id', 'postcode', 'state', 'gender', 'max(fraud_probability)')
consumer.show()

+-------+-----------+--------+-----+-----------+----------------------+
|user_id|consumer_id|postcode|state|     gender|max(fraud_probability)|
+-------+-----------+--------+-----+-----------+----------------------+
|      6|     407340|    2033|  NSW|     Female|    10.459280127078758|
|      7|     511685|    4606|  QLD|     Female|                   0.0|
|      9|     650435|    2482|  NSW|     Female|     10.58055311139687|
|      9|     650435|    2482|  NSW|     Female|     10.58055311139687|
| 142344|     389526|    1143|  NSW|       Male|                   0.0|
| 142348|     553507|    4723|  QLD|Undisclosed|                   0.0|
| 142352|    1220814|    6479|   WA|     Female|                   0.0|
| 142354|     338380|    3138|  VIC|       Male|                   0.0|
| 142357|     746099|    3226|  VIC|     Female|                   0.0|
| 142359|     895156|    3173|  VIC|     Female|                   0.0|
| 284819|     562577|    2175|  NSW|       Male|                

In [58]:
consumer = consumer.sort('user_id')

In [59]:
consumer.show()

+-------+-----------+--------+-----+-----------+----------------------+
|user_id|consumer_id|postcode|state|     gender|max(fraud_probability)|
+-------+-----------+--------+-----+-----------+----------------------+
|      1|    1195503|    6935|   WA|     Female|     9.805431136520959|
|      2|     179208|    2782|  NSW|     Female|    10.069850934775245|
|      2|     179208|    2782|  NSW|     Female|    10.069850934775245|
|      3|    1194530|     862|   NT|     Female|     8.300636455314633|
|      4|     154128|    2780|  NSW|     Female|     9.633302411090419|
|      5|     712975|    6355|   WA|     Female|    27.496186536467164|
|      5|     712975|    6355|   WA|     Female|    27.496186536467164|
|      5|     712975|    6355|   WA|     Female|    27.496186536467164|
|      6|     407340|    2033|  NSW|     Female|    10.459280127078758|
|      7|     511685|    4606|  QLD|     Female|                   0.0|
|      8|     448088|    6056|   WA|       Male|                

In [60]:
consumer.write.mode('overwrite').parquet('../data/curated/customer/')